# Zpracování a prezentace dat

Notebook pro získání moderátorů a respondentů zastoupených ve vybraných pořadech vysílání Českého rozhlasu.  

## Knihovny

In [ ]:
import csv
import math
import os

import pandas as pd
import requests
import cro.schedule as schedule
from pathlib import Path


DATA = Path("./data")

# import sys
#!{sys.executable} -m pip install scrapy
# import sys; sys.path
# import numpy; numpy.__path__

## Konfigurace

Od uživatele chceme aby zadal požadované období (dny) pro které budeme provádět analýzu.

### Vstup od uživatele

In [ ]:
dates = [f"2023-04-{i}" for i in range(1, 30)]

print(dates)

## Zpracování

### Získej program pro stanici Plus a Radiožurnál

In [ ]:
client = schedule.Client("plus")
schedules_ps = list(client.get_day_schedule(date) for date in dates)

In [ ]:
client = schedule.Client("radiozurnal")
schedules_rz = list(client.get_day_schedule(date) for date in dates)

In [ ]:
schedules = schedules_rz + schedules_ps

for s in schedules:
    print(s)

### Vyber jen požadované pořady

In [ ]:
schedule_data = []

for schedule in schedules:
    for index, show in enumerate(schedule.shows):
        if len(show.moderators) == 0:
            moderators = ""
        else:
            moderators = ";".join(p.name for p in show.moderators)
        relation_type = "repríza" if show.repetition else "premiéra"

        line = dict(
            station_name=show.station.name,
            show_type=show.type,
            show_kind_name=show.kind.name,
            show_description=show.description,
            show_id=show.id,
            show_title=show.title,
            show_date=show.date,
            show_since=show.since,
            show_till=show.till,
            relation_type=relation_type,
            show_duration_minutes=math.ceil(show.duration / 60),
            moderators=moderators,
        )

        schedule_data.append(line)

### Ulož program do CSV pro další zpracování

In [ ]:
# Uložíme CSV pro další zpracování.
with open(DATA / "schedule.csv", mode="w") as file:
    writer = csv.DictWriter(file, fieldnames=schedule_data[0].keys())
    writer.writeheader()
    writer.writerows(schedule_data)

---
**ZAČNI ZDE, POKUD CHCEŠ PRACOVAT S JIŽ ULOŽENÝM PROGRAMEM**

In [ ]:
SHOWS_WANTED = (
    "Pro a proti",
    "Interview Plus",
    "Rozhovory a komentáře",
    "Dvacet minut Radiožurnálu",
    "Hlavní zprávy - rozhovory a komentáře",
    "Speciál",
)

### Načti program a získej premiéry

- Načteme data do tabulky a aplikujeme voodoo a černou magii.
  - Vybereme pouze premiéry pořadů.
- Respondenty získáme pomocí parsování a služby Geneea (*entity recognition*).


In [ ]:
df = pd.read_csv(DATA / "schedule.csv")
df = df[df.relation_type == "premiéra"] # FIXME

### Ukaž unikátní názvy pořadů 

In [ ]:
shows_ps = set(df[df["station_name"] == "Plus"]["show_title"].to_list())
shows_rz = set(df[df["station_name"] == "Radiožurnál"]["show_title"].to_list())

print("POŘADY STANICE PLUS:\n", shows_ps)
print("POŘADY STANICE RADIOŽURNÁL:\n", shows_rz)

In [ ]:
# Vyber pouze sledované pořady
df = df[df["show_title"].isin(SHOWS_WANTED)] # FIXME
df.head(100)

## Kontrola

Zkontrolujeme si, jestli máme očekávaný počet premiér pořadů (nyní za měsíc):
- Dvacet minut Radiožurnálu: 23 (premiér/březen) * 2 (stanic) = 46 premiér / březen (23 na Radiožurnálu)
- Interview Plus: 23 (premiér/březen) * 1 (stanic) = 23 premiér / březen
- Pro a proti: 23 (premiér/březen) * 1 (stanic) = 23 premiér / březen
- Hlavní zprávy - rozhovory a komentáře: 23 * 2 (dvakrát za prac. den) * 2 (stanic) = 92 (46 na Radiožurnálu)


In [ ]:
# Kolik máme řádků/premiér pořadů (epizod).
len(df)

In [ ]:
#  Na Plusu se premiéry objevují jen pro dva dny v týdnu, zřejmě můžeme Plus úplně vyfiltrovat. 
df = df[~((df["show_title"] == "Dvacet minut Radiožurnálu") & (df["station_name"] == "Plus")) ]
df = df[~((df["show_title"] == "Hlavní zprávy - rozhovory a komentáře") & (df["station_name"] == "Plus")) ]

print("Očekávaný počet premiér 'Dvacet minut Radiožurnálu' odpovídá:",  len( df[df["show_title"] == "Dvacet minut Radiožurnálu"]) == 23)
print("Očekávaný počet premiér 'Interview Plus' odpovídá:",  len( df[df["show_title"] == "Interview Plus"]) == 23)
print("Očekávaný počet premiér 'Pro a proti' odpovídá:",  len( df[df["show_title"] == "Pro a proti"]) == 23)
print("Očekávaný počet premiér 'Hlavní zprávy - rozhovory a komentáře' odpovídá:",  len( df[df["show_title"] == "Hlavní zprávy - rozhovory a komentáře"]) == 46, len( df[df["show_title"] == "Hlavní zprávy - rozhovory a komentáře"]))

# df[df["show_title"] ==  "Hlavní zprávy - rozhovory a komentáře" ].head(100)

### Získej popis epizod pro NLP analýzu

In [ ]:
index_description_df = df[["show_title", "show_description"]]

#### Ukázka záznamů

In [ ]:
index_description_df.head(5)

#### Počet záznamů

In [ ]:
len(index_description_df)

### Vytvoř objekty pro NLP analýzu

Po potřeby Geneea Media V2 API vytvoříme JSON objekty, které zasíláme v dotazu. Jako index (atribut `id`) použijeme index řádku datového rámce, ten pak dostaneme po zpracování zpět. Tím můžeme získané entity opět spojit s původními řádky datového rámce. Pro jeden analyzovaný text můžeme dostat 0 až N rozpoznaných osob. Nekteré z nich mohou být moderátoři, ty se snažíme odfiltrovat pomocí daného seznamu.

In [ ]:
inputs = []
for i, record in index_description_df.iterrows():
    inputs.append(
        {
            "id": i,
            "paraSpecs": [{"type": "text", "text": record.show_description}],
        }
    )

# Uložíme si vstupy pro přehled.
# with open(DATA / "test.txt", mode="w", encoding="utf8") as f:
    # f.write("\n".join([str(i) for i in inputs]))

In [ ]:
BASE_URL = "https://media-api.geneea.com/v2/"
HEADERS = {"content-type": "application/json", "X-API-Key": os.getenv("GENEEA_API_KEY")}

results = [requests.post(f"{BASE_URL}nlp/analyze", json=input, headers=HEADERS).json() for input in inputs]

In [ ]:
len(results) == len(index_description_df)

### Uložíme  získané entity/osoby spolu s původním indexem záznamu 

Rozpoznaná entita má buď typ `person` nebo v některých případech typ `other`. To pak musíme hledat klíč `feats` a v něm dále `detectedType`. 

**Klíče nemusí existovat!**

In [ ]:
index_person = []
for result in results:
    if "entities" in result:
        for entity in result["entities"]:
            if entity["type"] == "person":
                index_person.append((result["id"], entity["stdForm"], entity["gkbId"]))
            if "feats" in entity:
                if "detectedType" == entity["feats"]:
                    if entity["feats"]["detectedType"] == "person":
                        index_person.append((result["id"], entity["stdForm"], entity["gkbId"]))

In [ ]:
respondents_df = pd.DataFrame(index_person, columns=["index", "respondent", "gkbid"])
respondents_df.set_index("index")
respondents_df.index.astype(int)
respondents_df

In [ ]:
# Vrátíme osoby do původního dataframu. Grupujeme si osoby podle indexu.
merged_respondents_df = respondents_df.groupby(["index"])
merged_respondents_df = merged_respondents_df['respondent'].apply(';'.join)
merged_respondents_df.reset_index().set_index("index")
merged_respondents_df = pd.DataFrame(merged_respondents_df)
merged_respondents_df

In [ ]:
merged_respondents_df.index = merged_respondents_df.index.astype(int)
print(merged_respondents_df.index.dtype)

# Iterace: Tohle je pěkná blbost z hlediska rychlosti, ale merge nějak nefungoval.
# Nicméně smyčka funguje a zjistili jsme, že indexy neměli správny typ, proto původní merge nefungoval.
# nechávám jako důkaz a připomínku vlastní blbosti.
# for row in df.iterrows():
#     if row[0] in merged_respondents_df.index:
#         print(row[0], merged_respondents_df.loc[row[0]])

In [ ]:
final_df = df.join(merged_respondents_df)

In [ ]:
final_df.to_csv(DATA / "final.csv")

### Zjisti funkce osob pomocí Geneea

**Nyní nemám přístup k této funkcionalitě.**

Zjistíme funkce pro všechny rozpoznané osoby. Prozatím negunguje / access denied

In [ ]:

ids = respondents_df["gkbid"].to_list()
data_input = {
    "ids": ids,
    "language": "cs",
}

result = requests.post(f'{BASE_URL}knowledgebase/infoboxes', json=data_input, headers=HEADERS)
result.content